In [1]:
import sys
import copy
import rospy
import moveit_commander
import moveit_msgs.msg
import geometry_msgs.msg
import time
import trajectory_msgs.msg 
import tf
import tf.transformations
from math import pi
from std_msgs.msg import String,Empty,UInt16
from moveit_commander.conversions import pose_to_list
import numpy as np
import helper
import math

In [2]:
moveit_commander.roscpp_initialize(sys.argv)
rospy.init_node('dual_arm_origami', anonymous=True)
listener = tf.TransformListener()
robot = moveit_commander.RobotCommander()
scene = moveit_commander.PlanningSceneInterface()
rospy.sleep(2)
group_name1 = "hong_arm"
group1 = moveit_commander.MoveGroupCommander(group_name1)
group_name2 = "kong_arm"
group2 = moveit_commander.MoveGroupCommander(group_name2)
display_trajectory_publisher = rospy.Publisher('/move_group/display_planned_path',
                                               moveit_msgs.msg.DisplayTrajectory,
                                               queue_size=20)

In [3]:
def addCollisionObjects():
    global box_pose1
    global box_name1
    box_pose1 = geometry_msgs.msg.PoseStamped()
    box_pose1.header.frame_id = "world"
    box_pose1.pose.orientation.w = 1.0
    box_pose1.pose.position.x = -0.105
    box_pose1.pose.position.y = -0.1485
    box_pose1.pose.position.z = 0.71
    box_name1 = 'box1'
    scene.add_box(box_name1, box_pose1, (0.01, 0.01, 0.01))
    global box_pose2
    global box_name2
    box_pose2 = geometry_msgs.msg.PoseStamped()
    box_pose2.header.frame_id = "world"
    box_pose2.pose.orientation.w = 1.0
    box_pose2.pose.position.x = 0.105
    box_pose2.pose.position.y = -0.1485
    box_pose2.pose.position.z = 0.71
    box_name2 = 'box2'
    scene.add_box(box_name2, box_pose2, (0.01, 0.01, 0.01))
    global box_pose3
    global box_name3
    box_pose3 = geometry_msgs.msg.PoseStamped()
    box_pose3.header.frame_id = "world"
    box_pose3.pose.orientation.w = 1.0
    box_pose3.pose.position.x = 0.105
    box_pose3.pose.position.y = 0.1485
    box_pose3.pose.position.z = 0.71
    box_name3 = 'box3'
    scene.add_box(box_name3, box_pose3, (0.01, 0.01, 0.01))
    global box_pose4
    global box_name4
    box_pose4 = geometry_msgs.msg.PoseStamped()
    box_pose4.header.frame_id = "world"
    box_pose4.pose.orientation.w = 1.0
    box_pose4.pose.position.x = -0.105
    box_pose4.pose.position.y = 0.1485
    box_pose4.pose.position.z = 0.71
    box_name4 = 'box4'
    scene.add_box(box_name4, box_pose4, (0.01, 0.01, 0.01))
    global paper_pose
    global paper_name
    paper_pose = geometry_msgs.msg.PoseStamped()
    paper_pose.header.frame_id = "world"
    paper_pose.pose.orientation.w = 1.0
    paper_pose.pose.position.x = 0
    paper_pose.pose.position.y = 0
    paper_pose.pose.position.z = 0.706
    paper_name = 'paper'
    scene.add_box(paper_name, paper_pose, (0.21, 0.297, 0.002))   
    global crease_pose1
    global crease_name1
    crease_pose1 = geometry_msgs.msg.PoseStamped()
    crease_pose1.header.frame_id = "world"
    crease_pose1.pose.orientation.x = 0
    crease_pose1.pose.orientation.y = 0
    crease_pose1.pose.orientation.z = -0.38268
    crease_pose1.pose.orientation.w = 0.9238777
    crease_pose1.pose.position.x = -0.03075
    crease_pose1.pose.position.y = -0.07425
    crease_pose1.pose.position.z = 0.7065
    crease_name1 = 'crease1'
    global crease_pose2
    global crease_name2
    crease_pose2 = geometry_msgs.msg.PoseStamped()
    crease_pose2.header.frame_id = "world"
    crease_pose2.pose.orientation.x = 0
    crease_pose2.pose.orientation.y = 0
    crease_pose2.pose.orientation.z = 0.38268
    crease_pose2.pose.orientation.w = 0.9238777
    crease_pose2.pose.position.x = -0.03075
    crease_pose2.pose.position.y = 0.07425
    crease_pose2.pose.position.z = 0.7065
    crease_name2 = 'crease2'    
    global crease_pose3
    global crease_name3
    crease_pose3 = geometry_msgs.msg.PoseStamped()
    crease_pose3.header.frame_id = "world"
#     crease_pose3.pose.orientation.x = 0
#     crease_pose3.pose.orientation.y = 0
#     crease_pose3.pose.orientation.z = 0.38268
    crease_pose3.pose.orientation.w = 1
    crease_pose3.pose.position.x = 0
    crease_pose3.pose.position.y = -0.07425
    crease_pose3.pose.position.z = 0.7065
    crease_name3 = 'crease3'    

In [4]:
def robkong_go_to_home(times):
    if times == 1:
        joint_values = [-1.64, -0.73, 1.2, -1.94, -1.64, 0.73]
    elif times == 2:
        joint_values = [-2.03, -1.1, 1.93, -2.42, -1.5, 1.23]
    group2.set_joint_value_target(joint_values)
    group2.go(joint_values, wait=True)
    group2.stop()
    group2.clear_pose_targets()

def robhong_go_to_home(times):
    if times == 1:
        joint_values = [-1.41, -1.49, 2.2, -2.28, -1.57, 0.16]
    elif times == 2:
        joint_values = [-2.04,-1.28,2.33,-2.69,-1.56,0.84]
    group1.set_joint_value_target(joint_values)
    group1.go(joint_values, wait=True)
    group1.stop()
    group1.clear_pose_targets()

In [5]:
def move_waypoints(dx,dy,dz,vel,robot_arm):
    if robot_arm == "robhong":
        group = group1
    elif robot_arm == "robkong":
        group = group2
    else:
        print "robot_arm input error, please input valid robot_arm name"
        return 0        
    waypoints = []
    waypoints.append(group.get_current_pose().pose)
    wpose = copy.deepcopy(group.get_current_pose().pose)
    wpose.position.x += dx
    wpose.position.y += dy
    wpose.position.z += dz
    waypoints.append(copy.deepcopy(wpose))
    (plan, fraction) = group.compute_cartesian_path(
                                        waypoints,   # waypoints to follow
                                        0.01,        # eef_step
                                        0.0)         # jump_threshold
    new_traj = scale_trajectory_speed(plan,vel)
    group.execute(new_traj)    
    
def rot_waypoints(initial_quat,target_quat,vel,robot_arm):
    if robot_arm == "robhong":
        group = group1
    elif robot_arm == "robkong":
        group = group2
    else:
        print "robot_arm input error, please input valid robot_arm name"
        return 0 
    pose_target = group.get_current_pose().pose
    waypoints = []
    for t in range(0, 4):
        quat_waypoints = tf.transformations.quaternion_slerp(initial_quat,target_quat,t/3)
        pose_target.orientation.x = quat_waypoints[0]
        pose_target.orientation.y = quat_waypoints[1]
        pose_target.orientation.z = quat_waypoints[2]
        pose_target.orientation.w = quat_waypoints[3]
        waypoints.append(copy.deepcopy(pose_target))
    (plan, fraction) = group.compute_cartesian_path(waypoints, 0.01, 0) # waypoints, resolution=1cm, jump_threshold)
    group.execute(plan)
    
def scale_trajectory_speed(traj, scale):
    new_traj = moveit_msgs.msg.RobotTrajectory()
    new_traj.joint_trajectory = traj.joint_trajectory
    n_joints = len(traj.joint_trajectory.joint_names)
    n_points = len(traj.joint_trajectory.points)
    points = list(traj.joint_trajectory.points)

    for i in range(n_points):
        point = trajectory_msgs.msg.JointTrajectoryPoint()
        point.time_from_start = traj.joint_trajectory.points[i].time_from_start / scale
        point.velocities = list(traj.joint_trajectory.points[i].velocities)
        point.accelerations = list(traj.joint_trajectory.points[i].accelerations)
        point.positions = traj.joint_trajectory.points[i].positions

        for j in range(n_joints):
            point.velocities[j] = point.velocities[j] * scale
            point.accelerations[j] = point.accelerations[j] * scale * scale
        points[i] = point

    new_traj.joint_trajectory.points = points
    return new_traj

In [6]:
def move_waypoints_temp(dx,dy,dz,initial_quat,target_quat,vel,robot_arm):
    if robot_arm == "robhong":
        group = group1
    elif robot_arm == "robkong":
        group = group2
    else:
        print "robot_arm input error, please input valid robot_arm name"
        return 0        
    waypoints = []
    pose_target = group.get_current_pose().pose
    waypoints.append(pose_target)
#     waypoints.append(copy.deepcopy(pose_target))
    for t in range(0, 4):
        pose_target.position.x += dx/3
        pose_target.position.y += dy/3
        pose_target.position.z += dz/3
        quat_waypoints = tf.transformations.quaternion_slerp(initial_quat,target_quat,t/3)
        pose_target.orientation.x = quat_waypoints[0]
        pose_target.orientation.y = quat_waypoints[1]
        pose_target.orientation.z = quat_waypoints[2]
        pose_target.orientation.w = quat_waypoints[3]
        waypoints.append(copy.deepcopy(pose_target))
    (plan, fraction) = group.compute_cartesian_path(
                                        waypoints,   # waypoints to follow
                                        0.01,        # eef_step
                                        0.0)         # jump_threshold
    new_traj = scale_trajectory_speed(plan,vel)
    group.execute(new_traj)    

In [7]:
def tilt(point, axis, angle, velocity, robot_arm):
    '''
    Parameters:
        point (list): 3-D coordinate of point in rotation axis
        axis (list): 3-D vector of rotation axis (right-hand rule)
        angle (double): angle of tilting
        velocity (double): robot velocity between 0 and 1
    '''
    if robot_arm == "robhong":
        group = group1
    elif robot_arm == "robkong":
        group = group2
    else:
        print "robot_arm input error, please input valid robot_arm name"
        return 0  
    # Normalize axis vector
    axis = axis / np.linalg.norm(axis)
    pose_target = group.get_current_pose().pose
    pos_initial = [pose_target.position.x, pose_target.position.y, pose_target.position.z]
    ori_initial = [pose_target.orientation.x, pose_target.orientation.y, pose_target.orientation.z, pose_target.orientation.w]

    # Tilt center point. Closest point from tcp to axis line
    center = np.add(point, np.dot(np.subtract(pos_initial, point), axis)*axis)

    # Closest distance from tcp to axis line
    radius = np.linalg.norm(np.subtract(center, pos_initial))

    # Pair of orthogonal vectors in tilt plane
    v1 =  -np.subtract(np.add(center, np.dot(np.subtract(pos_initial, center), axis)*axis), pos_initial)
    v1 = v1/np.linalg.norm(v1)
    v2 = np.cross(axis, v1)

    # Interpolate orientation poses via quaternion slerp
    q = helper.axis_angle2quaternion(axis, angle)
    ori_target = tf.transformations.quaternion_multiply(q, ori_initial)
    ori_waypoints = helper.slerp(ori_initial, ori_target, np.arange(1.0/angle , 1.0+1.0/angle, 1.0/angle))

    waypoints = []
    for t in range(1, angle + 1):
        circle = np.add(center, radius*(math.cos(math.radians(t)))*v1 + radius*(math.sin(math.radians(t)))*v2)
        pose_target.position.x = circle[0]
        pose_target.position.y = circle[1]
        pose_target.position.z = circle[2]
        pose_target.orientation.x = ori_waypoints[t-1][0]
        pose_target.orientation.y = ori_waypoints[t-1][1]
        pose_target.orientation.z = ori_waypoints[t-1][2]
        pose_target.orientation.w = ori_waypoints[t-1][3]
        waypoints.append(copy.deepcopy(pose_target))
    (plan, fraction) = group.compute_cartesian_path(waypoints, 0.01, 0) # waypoints, resolution=1cm, jump_threshold)
    plan = scale_trajectory_speed(plan,velocity)
    group.execute(plan)

In [8]:
def fix_paper(paper_edge_point1,paper_edge_point2,robot_arm):
    '''
    This function is  used for fixing paper, it has 2 steps:rotate to fix orientation, move to fix position.
    orientation: y axis of source_frame will be aligned with [point2 -> point1]
    position: origin of source_frame will be located at the mid of point1 and point2
    '''
    if robot_arm == "robhong":
        group = group1
        source_frame = "soft_gripper_hong"
        target_frame = "hong_tool0"
        lowest_frame = "rigid_tip_link1" #according to real condition, modify later in experiment
    elif robot_arm == "robkong":
        group = group2
        source_frame = "soft_gripper_kong"
        target_frame = "kong_tool0"
        lowest_frame = "rigid_tip_link2"
    else:
        print "robot_arm input error, please input valid robot_arm name"
        return 0
    #step1:rotate to the prefix oritation
    paper_axis = np.subtract(paper_edge_point1,paper_edge_point2).tolist()
    paper_axis = paper_axis / np.linalg.norm(paper_axis)
    paper_axis = paper_axis.tolist()
    x_axis = [0,0,-1]
    z_axis = np.cross(x_axis, paper_axis).tolist()
    y_axis = paper_axis
    x_axis.append(0)
    y_axis.append(0)
    z_axis.append(0)

    target_rot_mat = np.array([x_axis,
                     y_axis,
                     z_axis,
                     [0,0,0,1]]).transpose()
    target_rot_mat = target_rot_mat.tolist()
    listener.waitForTransform(source_frame,target_frame, rospy.Time(), rospy.Duration(4.0))
    (trans,rot) = listener.lookupTransform(source_frame, target_frame, rospy.Time(0))
    rot = tf.transformations.quaternion_matrix(rot)
    target_rot_mat = np.dot(target_rot_mat,rot).tolist()
    target_quat = tf.transformations.quaternion_from_matrix(target_rot_mat)
    s =group.get_current_pose().pose.orientation
    initial_quat = [s.x,s.y,s.z,s.w]
    rot_waypoints(initial_quat,target_quat,0.4,robot_arm)
    rospy.sleep(1)
    #step2:move to the fix position
    paper_edge_mid = np.subtract(paper_edge_point1,paper_edge_point2)*0.5 + np.array(paper_edge_point2)
    paper_edge_mid = paper_edge_mid.tolist()
    listener.waitForTransform("world",source_frame, rospy.Time(), rospy.Duration(4.0))
    (trans1,rot1) = listener.lookupTransform("world",source_frame, rospy.Time(0))
    listener.waitForTransform( "world",target_frame,rospy.Time(), rospy.Duration(4.0))
    (trans2,rot2) = listener.lookupTransform("world",target_frame,rospy.Time(0))
    trans_world_gripper = np.subtract(trans1,trans2).tolist()    

    listener.waitForTransform("world",lowest_frame, rospy.Time(), rospy.Duration(4.0))
    (trans_low,rot_low) = listener.lookupTransform("world",lowest_frame, rospy.Time(0))
    descend_height = trans_low[2]-0.72 #0.705 is the height of movable table
    current_pos = group.get_current_pose()

    delta = [paper_edge_mid[0]-current_pos.pose.position.x-trans_world_gripper[0],
            paper_edge_mid[1]-current_pos.pose.position.y-trans_world_gripper[1],
            -descend_height]
    move_waypoints(delta[0],delta[1],delta[2],0.4,robot_arm)

In [9]:
def flex_flip(paper_edge_point,paper_edge_axis,robot_arm):
    if robot_arm == "robhong":
        group = group1
        source_frame = "soft_gripper_hong"
        target_frame = "hong_tool0"
        lowest_frame = "rigid_tip_link1" #according to real condition, modify later in experiment
    elif robot_arm == "robkong":
        group = group2
        source_frame = "soft_gripper_kong"
        target_frame = "kong_tool0"
        lowest_frame = "rigid_tip_link2" #according to real condition, modify later in experiment
    else:
        print "robot_arm input error, please input valid robot_arm name"
        return 0
    #step1: rotate to preflex orientation
    paper_edge_axis = paper_edge_axis / np.linalg.norm(paper_edge_axis)
    paper_edge_axis = paper_edge_axis.tolist()
    x_axis = [0,0,-1]
    z_axis = paper_edge_axis
    y_axis = np.cross(z_axis,x_axis).tolist()
    z_axis = paper_edge_axis
    x_axis.append(0)
    y_axis.append(0)
    z_axis.append(0)

    rot_mat_temp = np.array([x_axis,
                     y_axis,
                     z_axis,
                     [0,0,0,1]]).transpose()
    rot_mat_temp = rot_mat_temp.tolist()
    rot_mat = tf.transformations.rotation_matrix(pi/4,(1,0,0))
    target_rot_mat = np.dot(rot_mat_temp,rot_mat).tolist()
    
    listener.waitForTransform(source_frame,target_frame, rospy.Time(), rospy.Duration(4.0))
    (trans_temp,rot_temp) = listener.lookupTransform(source_frame, target_frame, rospy.Time(0))
    rot_temp = tf.transformations.quaternion_matrix(rot_temp)
    target_rot_mat = np.dot(target_rot_mat,rot_temp).tolist()
    target_quat = tf.transformations.quaternion_from_matrix(target_rot_mat)
    s =group.get_current_pose().pose.orientation
    initial_quat = [s.x,s.y,s.z,s.w]
    rot_waypoints(initial_quat,target_quat,0.4,robot_arm)
    rospy.sleep(1)    
    #step2: soft tip link go to edge point
    listener.waitForTransform("world",source_frame, rospy.Time(), rospy.Duration(4.0))
    (trans1,rot1) = listener.lookupTransform("world",source_frame, rospy.Time(0))
    listener.waitForTransform( "world",target_frame,rospy.Time(), rospy.Duration(4.0))
    (trans2,rot2) = listener.lookupTransform("world",target_frame,rospy.Time(0))
    trans_world_gripper = np.subtract(trans1,trans2).tolist()
    current_pos = group.get_current_pose()
    listener.waitForTransform("world",lowest_frame, rospy.Time(), rospy.Duration(4.0))
    (trans_low,rot_low) = listener.lookupTransform("world",lowest_frame, rospy.Time(0))
    descend_height = trans_low[2]-0.715 #0.705 is the height of movable table
    delta = [paper_edge_point[0]-current_pos.pose.position.x-trans_world_gripper[0],
            paper_edge_point[1]-current_pos.pose.position.y-trans_world_gripper[1],
            -descend_height]
    move_waypoints(delta[0],delta[1],delta[2],0.4,robot_arm)
    #step3: rotate to leave space for gripper closing
    listener.waitForTransform("world",source_frame, rospy.Time(), rospy.Duration(4.0))
    (trans,rot) = listener.lookupTransform("world",source_frame, rospy.Time(0))
    center = trans[0:3] #center locates at soft gripper tip
    phi = 15
    rot = tf.transformations.quaternion_matrix(rot).tolist()
    axis = np.dot(rot,[0,0,-1,1]).tolist()[0:3] #z_axis of soft_gripper_$(robot_arm) frame
    tilt(center,axis,phi,0.5,robot_arm)
    listener.waitForTransform("world",lowest_frame, rospy.Time(), rospy.Duration(4.0))
    (trans_low,rot_low) = listener.lookupTransform("world",lowest_frame, rospy.Time(0))
    descend_height = trans_low[2]-0.715 #0.705 is the height of movable table
    move_waypoints(0,0,-descend_height,0.4,robot_arm)
    
def fold(crease_point,crease_axis,fold_angle,move_dist,robot_arm,vel=0.4):
    #step1: rotate around the crease line
    tilt(crease_point,crease_axis,fold_angle,0.5,robot_arm)
    #step2: move to the target position
    x_axis = [0,0,-1]
    crease_axis = crease_axis / np.linalg.norm(crease_axis)
    direction = np.cross(x_axis,crease_axis)
    direction = direction*move_dist
    move_waypoints(direction[0],direction[1],direction[2],0.4,robot_arm)

In [10]:
def scooping(scooping_point,scooping_axis,scooping_angle,robot_arm,scooping_dis=0.04):
    '''
    This function is used for scooping paper, has 3 steps:rotate to scooping orientation, move to scooping position,
        and move towards scooping direction at the distance of scooping_dis to scoop paper.
    Orientation: y axis of source_frame will be aligned with scooping_axis, scooping_angle (rad) is the angle between 
        ground and rigid finger of robot_arm.
    Position: origin of source_frame will be located at scooping point.
    Direction: according to right-hand rule, cross product of ([0,0,-1],scooping_axis) in world
    ''' 
    if robot_arm == "robhong":
        group = group1
        source_frame = "rigid_tip_link1"
        target_frame = "hong_tool0"
    elif robot_arm == "robkong":
        group = group2
        source_frame = "rigid_tip_link2"
        target_frame = "kong_tool0"
    else:
        print "robot_arm input error, please input valid robot_arm name"
        return 0
    # step1: rotate to scooping orientation
    scooping_axis = scooping_axis / np.linalg.norm(scooping_axis)
    scooping_axis = scooping_axis.tolist()    
    x_axis = [0,0,-1]
    z_axis = np.cross(x_axis, scooping_axis).tolist()
    y_axis = scooping_axis
    x_axis.append(0)
    y_axis.append(0)
    z_axis.append(0)

    rot_mat_temp = np.array([x_axis,
                     y_axis,
                     z_axis,
                     [0,0,0,1]]).transpose()
    rot_mat_temp = rot_mat_temp.tolist()
    #tilt_angle: rad, i.e. pi/4
    rot_mat = tf.transformations.rotation_matrix(scooping_angle,(0,1,0))
    target_rot_mat = np.dot(rot_mat_temp,rot_mat).tolist()
    
    listener.waitForTransform(source_frame,target_frame, rospy.Time(), rospy.Duration(4.0))
    (trans,rot) = listener.lookupTransform(source_frame, target_frame, rospy.Time(0))
    rot = tf.transformations.quaternion_matrix(rot)
    target_rot_mat = np.dot(target_rot_mat,rot).tolist()
    target_quat = tf.transformations.quaternion_from_matrix(target_rot_mat)
    s =group.get_current_pose().pose.orientation
    initial_quat = [s.x,s.y,s.z,s.w]
    rot_waypoints(initial_quat,target_quat,0.4,robot_arm)
    rospy.sleep(1)
    # step2: move to scooping position
    listener.waitForTransform("world",source_frame, rospy.Time(), rospy.Duration(4.0))
    (trans1,rot1) = listener.lookupTransform("world",source_frame, rospy.Time(0))
    listener.waitForTransform( "world",target_frame,rospy.Time(), rospy.Duration(4.0))
    (trans2,rot2) = listener.lookupTransform("world",target_frame,rospy.Time(0))
    trans_world_tip = np.subtract(trans1,trans2).tolist()    
    descend_height = trans1[2]-0.72 #0.705 is the height of movable table
    current_pos = group.get_current_pose()
    delta = [scooping_point[0]-current_pos.pose.position.x-trans_world_tip[0],
            scooping_point[1]-current_pos.pose.position.y-trans_world_tip[1],
            -descend_height]
    move_waypoints(delta[0],delta[1],delta[2],0.4,robot_arm)
    rospy.sleep(1)
    # step3: move towards scooping direction
    direction = np.array(z_axis[0:3])
    direction = direction*scooping_dis
    move_waypoints(direction[0],direction[1],direction[2],0.4,robot_arm)

In [11]:
def make_crease(paper_edge_point1,paper_edge_point2,robot_arm,n=5):
    '''
    This function is used for making a crease, has 2 steps:rotate to crease-making orientation, 
        move to crease-making position.
    Orientation: y axis of source_frame will be aligned with [point2 -> point1]
    Position: origin of source_frame will be located at point2
    '''
    if robot_arm == "robhong":
        group = group1
        source_frame = "rigid_tip_link1"
        target_frame = "hong_tool0"
    elif robot_arm == "robkong":
        group = group2
        source_frame = "rigid_tip_link2"
        target_frame = "kong_tool0"
    else:
        print "robot_arm input error, please input valid robot_arm name"
        return 0
    #step1: rotate to crease-making orientation
    paper_axis = np.subtract(paper_edge_point1,paper_edge_point2).tolist()
    paper_axis = paper_axis / np.linalg.norm(paper_axis)
    paper_axis = paper_axis.tolist()
    x_axis = [0,0,-1]
    z_axis = np.cross(x_axis, paper_axis).tolist()
    y_axis = paper_axis
    x_axis.append(0)
    y_axis.append(0)
    z_axis.append(0)

    rot_mat_temp = np.array([x_axis,
                     y_axis,
                     z_axis,
                     [0,0,0,1]]).transpose()
    rot_mat_temp = rot_mat_temp.tolist()
    rot_mat = tf.transformations.rotation_matrix(pi/4,(0,1,0))
    target_rot_mat = np.dot(rot_mat_temp,rot_mat).tolist()
    
    listener.waitForTransform(source_frame,target_frame, rospy.Time(), rospy.Duration(4.0))
    (trans,rot) = listener.lookupTransform(source_frame, target_frame, rospy.Time(0))
    rot = tf.transformations.quaternion_matrix(rot)
    target_rot_mat = np.dot(target_rot_mat,rot).tolist()
    target_quat = tf.transformations.quaternion_from_matrix(target_rot_mat)
    s =group.get_current_pose().pose.orientation
    initial_quat = [s.x,s.y,s.z,s.w]
    rot_waypoints(initial_quat,target_quat,0.4,robot_arm)
    rospy.sleep(1)
    #step2: move to crease-making position
    interval = np.subtract(paper_edge_point2,paper_edge_point1) / n
    listener.waitForTransform("world",source_frame, rospy.Time(), rospy.Duration(4.0))
    (trans1,rot1) = listener.lookupTransform("world",source_frame, rospy.Time(0))
    listener.waitForTransform( "world",target_frame,rospy.Time(), rospy.Duration(4.0))
    (trans2,rot2) = listener.lookupTransform("world",target_frame,rospy.Time(0))
    trans_world_tip = np.subtract(trans1,trans2).tolist()    
    descend_height = trans1[2]-0.72 #0.705 is the height of movable table
    move_waypoints(0,0,-descend_height,0.4,robot_arm)
    for i in range(n+1):
        point = np.array(paper_edge_point1) + i*interval
        point = point.tolist()
        current_pos = group.get_current_pose()
        delta = [point[0]-current_pos.pose.position.x-trans_world_tip[0],
                point[1]-current_pos.pose.position.y-trans_world_tip[1],
                0]
        move_waypoints(delta[0],delta[1],delta[2],0.4,robot_arm) 
        rospy.sleep(1)
        direction = np.array(z_axis[0:3])
        direction = direction*0.02
        move_waypoints(direction[0],direction[1],direction[2],0.4,robot_arm)
        rospy.sleep(1)
        move_waypoints(-direction[0],-direction[1],-direction[2],0.4,robot_arm)

In [12]:
# scene.add_box(crease_name1, crease_pose1, (0.21, 0.01, 0.005))
# scene.add_box(crease_name2, crease_pose2, (0.21, 0.01, 0.005))
scene.remove_world_object(crease_name1)
scene.remove_world_object(crease_name2)
scene.remove_world_object(crease_name3)
# scene.add_box(crease_name3, crease_pose3, (0.21, 0.01, 0.005))
# robkong_go_to_home(2)
# robhong_go_to_home(2)
# fix_paper(point1,point2,"robkong")
# flex_flip(point4,edge4,"robhong")
# fold(point3,crease_axis2,30,0.1, "robhong")
# move_waypoints(0.2,0.1,0,0.4,"robkong")
# scene.add_box(crease_name2, crease_pose2, (0.21, 0.01, 0.005))
# make_crease(crease_point3,crease_point1,"robkong")
# scene.add_box(crease_name2, crease_pose2, (0.21, 0.01, 0.005))
# fix_paper(point1,point2,"robhong")
# flex_flip(point4,edge4,"robkong")
# fold(point3,crease_axis2,30,0.05, "robkong")

NameError: name 'crease_name1' is not defined

In [13]:
# We can get the name of the reference frame for this robot:
planning_frame = group1.get_planning_frame()
print "============ Reference frame1: %s" % planning_frame
eef_link = group1.get_end_effector_link()
print "============ End effector1: %s" % eef_link
planning_frame = group2.get_planning_frame()
print "============ Reference frame2: %s" % planning_frame
eef_link = group2.get_end_effector_link()
print "============ End effector2: %s" % eef_link
group_names = robot.get_group_names()
print "============ Robot Groups:", robot.get_group_names()

============ Reference frame1: /world
============ End effector1: hong_wrist_3_link
============ Reference frame2: /world
============ End effector2: kong_wrist_3_link
============ Robot Groups: ['hong_arm', 'hong_hand', 'kong_arm', 'kong_hand']


In [14]:
#_____INITIALIZATION_____#
addCollisionObjects()
rospy.sleep(1)
point1 = [box_pose1.pose.position.x,box_pose1.pose.position.y,box_pose1.pose.position.z]
point2 = [box_pose2.pose.position.x,box_pose2.pose.position.y,box_pose2.pose.position.z]
point3 = [box_pose3.pose.position.x,box_pose3.pose.position.y,box_pose3.pose.position.z]
point4 = [box_pose4.pose.position.x,box_pose4.pose.position.y,box_pose4.pose.position.z]
edge1 = np.subtract(point1,point2).tolist()
edge2 = np.subtract(point2,point3).tolist()
edge3 = np.subtract(point3,point4).tolist()
edge4 = np.subtract(point4,point1).tolist()
crease_axis1 = [-0.7071068,0.7071068,0]
crease_axis2 = [0.7071068,0.7071068,0]
crease_axis3 = [-1,0,0]
crease_point1 = [-0.105,0,0.76]
crease_point2 = [0.0435,-0.1485,0.76]
crease_point3 = [0.0435,0.1485,0.76]
crease_point4 = [0.105,-0.07425,0.76]
crease_point5 = [-0.015375,-0.07425,0.76]

In [16]:
robkong_go_to_home(1)
robhong_go_to_home(1)

In [88]:
fix_paper(point3,point4,"robhong")

In [89]:
flex_flip(point1,edge1,"robkong")

In [90]:
fold(point2, crease_axis1,30,0.1, "robkong")

In [91]:
scene.add_box(crease_name1, crease_pose1, (0.21, 0.01, 0.005))
make_crease(crease_point1,crease_point2,"robhong",2)

In [92]:
fix_paper(point3,point4,"robkong")

In [93]:
scooping(crease_point2,crease_axis3,pi/4,"robhong")

In [94]:
fold(crease_point4, crease_axis3,60,0.1, "robhong",0.08)

In [95]:
scene.add_box(crease_name3, crease_pose3, (0.21, 0.01, 0.005))
make_crease(crease_point5,crease_point4,"robkong",2)

In [100]:
robkong_go_to_home(2)
fix_paper(point3,point4,"robkong")

In [97]:
#solution 1: generate a part of trajectory by us to avoid collsion
move_waypoints(0.35,0,0,0.4,"robkong") #first move in the x direction
move_waypoints(0,-0.35,0,0.4,"robkong") #second move in the y direction
rospy.sleep(1)
make_crease(crease_point5,crease_point4,"robkong",2)

In [99]:
#solution 2: generate the whole trajectory in moveit to avoid collision
pose_goal = geometry_msgs.msg.Pose()
pose_goal.orientation.x = -0.66976218
pose_goal.orientation.y = 0.6697671
pose_goal.orientation.z = 0.22675032
pose_goal.orientation.w = 0.22674866
pose_goal.position.x = -0.0751493723373
pose_goal.position.y = -0.173646802403
pose_goal.position.z = 0.833043830035
group2.set_pose_target(pose_goal)
group2.go(wait=True)
group2.clear_pose_targets()